In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
from datetime import datetime

In [2]:
# Categorías principales
categorias = [
    "La Ciudad", "Cultura y Espectáculos", "Educación", "Aire Libre", "Por Distrito", "Semana Santa"
]

# Subcategorías
subcategorias = {
    "La Ciudad": ["Actividades", "Carnaval", "Centros de Atención a Las Familias", "Cuentacuentos", "Navidad", "Planes por Distritos", "Rocódromos", "Semana Santa"],
    "Cultura y Espectáculos": ["Cine Infantil", "Circo", "Conciertos", "Exposiciones", "Monumentos", "Museos para niños", "Proyecciones", "Talleres", "Teatro Infantil Madrid"],
    "Educación": ["Ayudas y Becas", "Campamentos Urbanos", "Ludotecas"],
    "Aire Libre": ["Actividades al Aire Libre", "Actividades con animales", "Parques y jardines"],
    "Por Distrito": ["Arganzuela", "Barajas", "Carabanchel", "Centro", "Chamartín", "Chamberí", "Ciudad Lineal", "Fuencarral-El Pardo", "Hortaleza", "La Latina", "Moncloa-Aravaca", "Moratalaz", "Puente de Vallecas", "Retiro", "Salamanca", "San Blas-Canillejas", "Tetúan", "Usera", "Vicálvaro", "Villa de Vallecas", "Villaverde"]
}

# Extraer eventos de la página
eventos = [
    {"titulo": "Procesiones Semana Santa 2025 Madrid", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "12/04/2025", "visitas": 956, "distrito": "Centro", "descripcion": "Guía para familias con ganas de tradición y planazo", "gratuito": True},
    {"titulo": "Saetas de Semana Santa en Madrid 2025", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "10/04/2025", "visitas": 220, "distrito": "Centro", "descripcion": "Emoción a flor de piel en cada balcón", "gratuito": True},
    {"titulo": "Domingo de Resurrección en Madrid", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "10/04/2025", "visitas": 219, "distrito": "Centro", "descripcion": "Una tamborrada para despertar emociones", "gratuito": True},
    {"titulo": "Exposición de Semana Santa en Madrid", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "10/04/2025", "visitas": 201, "distrito": "Centro", "descripcion": "Una tradición con alma", "gratuito": True},
    {"titulo": "Conciertos gratuitos de órgano y banda sinfónica por Semana Santa 2025", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "10/04/2025", "visitas": 1200, "distrito": "Centro", "descripcion": "Música clásica para Semana Santa", "gratuito": True},
    {"titulo": "Ruta de la Torrija Madrid 2025", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "10/04/2025", "visitas": 719, "distrito": "Centro", "descripcion": "Un viaje dulce por los sabores de Semana Santa", "gratuito": False},
    {"titulo": "8 planes para disfrutar en familia: Semana Santa con niños en Madrid", "categoria": "Semana Santa", "subcategoria": "Semana Santa", "fecha": "08/04/2025", "visitas": 3800, "distrito": "Varios", "descripcion": "Selección de planes para Semana Santa", "gratuito": None},
    {"titulo": "El Rey León: exposición inmersiva en Madrid", "categoria": "Cultura y Espectáculos", "subcategoria": "Exposiciones", "fecha": "15/04/2025", "visitas": 21, "distrito": "Centro", "descripcion": "Exposición que hará rugir a toda la familia", "gratuito": False},
    {"titulo": "Campamento de verano del Price: Magia en Madrid", "categoria": "Educación", "subcategoria": "Campamentos Urbanos", "fecha": "15/04/2025", "visitas": 21, "distrito": "Centro", "descripcion": "Magia en Madrid para tus peques", "gratuito": False},
    {"titulo": "Campamento de verano Faunia", "categoria": "Educación", "subcategoria": "Campamentos Urbanos", "fecha": "15/04/2025", "visitas": 20, "distrito": "Vicálvaro", "descripcion": "Una aventura salvajemente divertida", "gratuito": False},
    {"titulo": "Talleres Circo Price", "categoria": "Cultura y Espectáculos", "subcategoria": "Talleres", "fecha": "13/04/2025", "visitas": 248, "distrito": "Centro", "descripcion": "Talleres para niños en el Circo Price", "gratuito": False},
    {"titulo": "Planetario de Madrid: Proyecciones en Abril 2025", "categoria": "Cultura y Espectáculos", "subcategoria": "Proyecciones", "fecha": "13/04/2025", "visitas": 513, "distrito": "Salamanca", "descripcion": "Proyecciones para toda la familia", "gratuito": False},
    {"titulo": "Teatro de Títeres en Retiro – Programación Abril 2025", "categoria": "Cultura y Espectáculos", "subcategoria": "Teatro Infantil Madrid", "fecha": "13/04/2025", "visitas": 379, "distrito": "Retiro", "descripcion": "Teatro de títeres para niños", "gratuito": True},
    {"titulo": "Programación infantil en Cineteca de Arganzuela: Abril 2025", "categoria": "Cultura y Espectáculos", "subcategoria": "Cine Infantil", "fecha": "13/04/2025", "visitas": 390, "distrito": "Arganzuela", "descripcion": "Cine para niños en Arganzuela", "gratuito": False},
    {"titulo": "Cine Infantil en Moratalaz – Abril 2025", "categoria": "Cultura y Espectáculos", "subcategoria": "Cine Infantil", "fecha": "13/04/2025", "visitas": 275, "distrito": "Moratalaz", "descripcion": "Cine para niños en Moratalaz", "gratuito": False},
    {"titulo": "Visitas guiadas al Centro Rainfer", "categoria": "Aire Libre", "subcategoria": "Actividades con animales", "fecha": "07/04/2025", "visitas": 446, "distrito": "Fuencarral-El Pardo", "descripcion": "Una aventura solidaria en familia", "gratuito": False},
    {"titulo": "¿Dónde ver los cerezos en flor en Madrid?", "categoria": "Aire Libre", "subcategoria": "Parques y jardines", "fecha": "17/03/2025", "visitas": 322, "distrito": "Varios", "descripcion": "Guía para ver los cerezos en flor", "gratuito": True},
    {"titulo": "El Tren de la Fresa", "categoria": "Aire Libre", "subcategoria": "Actividades al Aire Libre", "fecha": "16/03/2025", "visitas": 636, "distrito": "Varios", "descripcion": "El viaje en Madrid que todo el mundo quiere hacer", "gratuito": False},
    {"titulo": "Cómo saber si un Parque de Madrid está abierto o cerrado", "categoria": "Aire Libre", "subcategoria": "Parques y jardines", "fecha": "14/03/2025", "visitas": 714, "distrito": "Varios", "descripcion": "Información sobre apertura de parques", "gratuito": True},
    {"titulo": "El Tren de Arganda", "categoria": "Aire Libre", "subcategoria": "Actividades al Aire Libre", "fecha": "03/03/2025", "visitas": 230, "distrito": "Arganda", "descripcion": "Tren histórico para familias", "gratuito": False},
    {"titulo": "Tren gratuito para descubrir el Parque Juan Carlos I", "categoria": "Aire Libre", "subcategoria": "Parques y jardines", "fecha": "28/02/2025", "visitas": 310, "distrito": "Hortaleza", "descripcion": "Tren gratuito por el parque", "gratuito": True},
    {"titulo": "Parques de Bolas y Áreas de Juego Indoor", "categoria": "La Ciudad", "subcategoria": "Actividades", "fecha": "22/03/2025", "visitas": 1200, "distrito": "Varios", "descripcion": "Espacios para Jugar a Cubierto en Madrid", "gratuito": False},
    {"titulo": "Rocódromos en Madrid", "categoria": "La Ciudad", "subcategoria": "Rocódromos", "fecha": "25/01/2025", "visitas": 114, "distrito": "Varios", "descripcion": "Guía de rocódromos para niños", "gratuito": False},
    {"titulo": "Grupos de Whatsapp para familias por Distrito en Madrid", "categoria": "La Ciudad", "subcategoria": "Planes por Distritos", "fecha": "10/01/2025", "visitas": 538, "distrito": "Varios", "descripcion": "Grupos de comunicación por distritos", "gratuito": True},
    {"titulo": "Centros de Apoyo a las Familias (CAF)", "categoria": "La Ciudad", "subcategoria": "Centros de Atención a Las Familias", "fecha": "02/02/2025", "visitas": 107, "distrito": "Varios", "descripcion": "Información sobre los CAF", "gratuito": True},
    {"titulo": "Cuenta cuentos en Madrid", "categoria": "La Ciudad", "subcategoria": "Cuentacuentos", "fecha": "07/01/2025", "visitas": 742, "distrito": "Varios", "descripcion": "Agenda de cuentacuentos", "gratuito": True},
    {"titulo": "Campamentos urbanos para Semana Santa 2025 en Madrid", "categoria": "Educación", "subcategoria": "Campamentos Urbanos", "fecha": "25/02/2025", "visitas": 1100, "distrito": "Varios", "descripcion": "Opciones de campamentos para Semana Santa", "gratuito": False},
    {"titulo": "Campamento urbano de Carnaval 2025", "categoria": "Educación", "subcategoria": "Campamentos Urbanos", "fecha": "09/02/2025", "visitas": 387, "distrito": "Varios", "descripcion": "Campamentos para Carnaval", "gratuito": False}
]

# Crear DataFrame de eventos
df_eventos = pd.DataFrame(eventos)

# Convertir fechas a formato datetime
df_eventos['fecha'] = pd.to_datetime(df_eventos['fecha'], format='%d/%m/%Y')

# Añadir columna de mes
df_eventos['mes'] = df_eventos['fecha'].dt.month_name()

# Añadir columna de año
df_eventos['año'] = df_eventos['fecha'].dt.year

# Ordenar por fecha (más reciente primero)
df_eventos = df_eventos.sort_values('fecha', ascending=False)

# Crear DataFrame de categorías
categorias_data = []
for cat in categorias:
    if cat in subcategorias:
        for subcat in subcategorias[cat]:
            categorias_data.append({'categoria': cat, 'subcategoria': subcat})
    else:
        categorias_data.append({'categoria': cat, 'subcategoria': None})

df_categorias = pd.DataFrame(categorias_data)

# Crear DataFrame de distritos
distritos = sorted(list(set(subcategorias["Por Distrito"])))
df_distritos = pd.DataFrame(distritos, columns=['distrito'])

# Unificar todos los datos en un solo archivo CSV
# Primero, añadimos un identificador a cada DataFrame
df_eventos['tipo_dato'] = 'evento'
df_categorias['tipo_dato'] = 'categoria'
df_distritos['tipo_dato'] = 'distrito'

# Completar columnas faltantes en cada DataFrame para poder unirlos
for col in df_eventos.columns:
    if col not in df_categorias.columns:
        df_categorias[col] = None
    if col not in df_distritos.columns:
        df_distritos[col] = None

# Unir los DataFrames
df_unificado = pd.concat([df_eventos, df_categorias, df_distritos], ignore_index=True)

# Guardar el DataFrame unificado
df_unificado.to_csv('planes_familiares_madrid_unificado.csv', index=False)

# Mostrar análisis básico
print("Dataset unificado creado con éxito.")
print("\nNúmero total de registros:", len(df_unificado))
print("\nDistribución por tipo de dato:")
print(df_unificado['tipo_dato'].value_counts())

# Análisis de eventos
df_solo_eventos = df_unificado[df_unificado['tipo_dato'] == 'evento']
print("\nAnálisis de eventos:")
print("Número total de eventos:", len(df_solo_eventos))
print("\nEventos por categoría:")
print(df_solo_eventos['categoria'].value_counts())
print("\nEventos por distrito:")
print(df_solo_eventos['distrito'].value_counts())
print("\nEventos gratuitos vs. de pago:")
print(df_solo_eventos['gratuito'].value_counts())

# Eventos más populares (por número de visitas)
print("\nTop 5 eventos más populares:")
print(df_solo_eventos.sort_values('visitas', ascending=False)[['titulo', 'visitas']].head(5))

# Crear un archivo CSV separado solo con eventos para facilitar el análisis
df_solo_eventos.to_csv('planes_familiares_madrid_eventos.csv', index=False)
print("\nArchivo de eventos creado por separado para facilitar el análisis.")

print("\nTodos los datos han sido unificados en 'planes_familiares_madrid_unificado.csv'")
print("Además, se ha creado un archivo separado 'planes_familiares_madrid_eventos.csv' con solo los eventos.")

Dataset unificado creado con éxito.

Número total de registros: 94

Distribución por tipo de dato:
tipo_dato
categoria    45
evento       28
distrito     21
Name: count, dtype: int64

Análisis de eventos:
Número total de eventos: 28

Eventos por categoría:
categoria
Semana Santa              7
Aire Libre                6
Cultura y Espectáculos    6
La Ciudad                 5
Educación                 4
Name: count, dtype: int64

Eventos por distrito:
distrito
Varios                 11
Centro                  9
Vicálvaro               1
Moratalaz               1
Retiro                  1
Arganzuela              1
Salamanca               1
Fuencarral-El Pardo     1
Arganda                 1
Hortaleza               1
Name: count, dtype: int64

Eventos gratuitos vs. de pago:
gratuito
False    15
True     12
Name: count, dtype: int64

Top 5 eventos más populares:
                                               titulo visitas
14  8 planes para disfrutar en familia: Semana San...    3800
16  

C:\Users\anoni\AppData\Local\Temp\ipykernel_14632\66098946.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_unificado = pd.concat([df_eventos, df_categorias, df_distritos], ignore_index=True)
